In [2]:
import collections
import functools

from absl import app
from absl import flags

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

import sys

sys.path.append("/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg")
print(sys.path)

import simple_fedavg_tff

['/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/tester/venv/lib/python3.9/site-packages', '/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg', '/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg', '/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg']


In [3]:
# Training hyperparameters
TOTAL_ROUNDS = 50 #NUMBER OF TOTAL TRAINING ROUNDS
ROUNDS_PER_EVAL = 1 #HOW OFTEN TO EVALUATE
TRAIN_CLIENTS_PER_ROUND = 2 #HOW MANY CLIENTS TO SAMPLE PER ROUND
CLIENT_EPOCHS_PER_ROUND = 1 #NUMBER OF EPOCHS IN THE CLIENT TO TAKE PER ROUND
BATCH_SIZE = 16 #BATCH SIZE USED ON THE CLIENT
TEST_BATCH_SIZE = 128 #MINIBATCH SIZE OF TEST DATA
SERVER_LEARNING_RATE = 1.0 #SERVER LEARNING RATE
CLIENT_LEARNING_RATE = 0.1 #CLIENT LEARNING RATE

In [4]:
def evaluate(keras_model, test_dataset):
  """Evaluate the acurracy of a keras model on a test dataset."""
  metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
  for batch in test_dataset:
    predictions = keras_model(batch['x'])
    metric.update_state(y_true=batch['y'], y_pred=predictions)
  return metric.result()


def get_emnist_dataset():
  """Loads and preprocesses the EMNIST dataset.

  Returns:
    A `(emnist_train, emnist_test)` tuple where `emnist_train` is a
    `tff.simulation.datasets.ClientData` object representing the training data
    and `emnist_test` is a single `tf.data.Dataset` representing the test data
    of all clients.
  """
  emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(
      only_digits=True)
  def element_fn(element):
    return collections.OrderedDict(
        x=tf.expand_dims(element['pixels'], -1), y=element['label'])

  def preprocess_train_dataset(dataset):
    # Use buffer_size same as the maximum client dataset size,
    # 418 for Federated EMNIST
    return dataset.map(element_fn).shuffle(buffer_size=418).repeat(
        count=CLIENT_EPOCHS_PER_ROUND).batch(
            BATCH_SIZE, drop_remainder=False)
  def preprocess_test_dataset(dataset):
    return dataset.map(element_fn).batch(
        TEST_BATCH_SIZE, drop_remainder=False)
  emnist_train = emnist_train.preprocess(preprocess_train_dataset)
  emnist_test = preprocess_test_dataset(
      emnist_test.create_tf_dataset_from_all_clients())
  return emnist_train, emnist_test


def create_original_fedavg_cnn_model(only_digits=True):
  """The CNN model used in https://arxiv.org/abs/1602.05629.

  Args:
    only_digits: If True, uses a final layer with 10 outputs, for use with the
      digits only EMNIST dataset. If False, uses 62 outputs for the larger
      dataset.

  Returns:
    An uncompiled `tf.keras.Model`.
  """
  data_format = 'channels_last'
  input_shape = [28, 28, 1]
  max_pool = functools.partial(
      tf.keras.layers.MaxPooling2D,
      pool_size=(2, 2),
      padding='same',
      data_format=data_format)
  conv2d = functools.partial(
      tf.keras.layers.Conv2D,
      kernel_size=5,
      padding='same',
      data_format=data_format,
      activation=tf.nn.relu)
  model = tf.keras.models.Sequential([
      conv2d(filters=32, input_shape=input_shape),
      max_pool(),
      conv2d(filters=64),
      max_pool(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dense(10 if only_digits else 62),
  ])
  return model


def server_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=SERVER_LEARNING_RATE)


def client_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=CLIENT_LEARNING_RATE)

In [5]:
def main(argv):
  if len(argv) > 1:
    raise app.UsageError('Too many command-line arguments.')
  # If GPU is provided, TFF will by default use the first GPU like TF. The
  # following lines will configure TFF to use multi-GPUs and distribute client
  # computation on the GPUs. Note that we put server computatoin on CPU to avoid
  # potential out of memory issue when a large number of clients is sampled per
  # round. The client devices below can be an empty list when no GPU could be
  # detected by TF.
  client_devices = tf.config.list_logical_devices('GPU')
  server_device = tf.config.list_logical_devices('CPU')[0]
  tff.backends.native.set_local_python_execution_context(
      server_tf_device=server_device, client_tf_devices=client_devices)
  train_data, test_data = get_emnist_dataset()

  def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""
    keras_model = create_original_fedavg_cnn_model(only_digits=True)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    return tff.learning.from_keras_model(
        keras_model,
        loss=loss,
        metrics=metrics,
        input_spec=train_data.element_type_structure)

  iterative_process = simple_fedavg_tff.build_federated_averaging_process(
      tff_model_fn, server_optimizer_fn, client_optimizer_fn)
  server_state = iterative_process.initialize()
  # Keras model that represents the global model we'll evaluate test data on.
  keras_model = create_original_fedavg_cnn_model(only_digits=True)
  for round_num in range(TOTAL_ROUNDS):
    sampled_clients = np.random.choice(
        train_data.client_ids,
        size=TRAIN_CLIENTS_PER_ROUND,
        replace=False)
    sampled_train_data = [
        train_data.create_tf_dataset_for_client(client)
        for client in sampled_clients
    ]
    server_state, train_metrics = iterative_process.next(
        server_state, sampled_train_data)
    print(f'Round {round_num}')
    print(f'\tTraining metrics: {train_metrics}')
    if round_num % ROUNDS_PER_EVAL == 0:
      server_state.model.assign_weights_to(keras_model)
      accuracy = evaluate(keras_model, test_data)
      print(f'\tValidation accuracy: {accuracy * 100.0:.2f}%')

Patch asyncio to allow nested event loops. If not used, runtime error in Jupyter NB

In [6]:
import nest_asyncio
nest_asyncio.apply()

In [7]:
try:
    app.run(main([None]))
except:
    print("exec end error")

2022-06-08 12:12:32.954908: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-08 12:12:32.954929: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 12:12:32.954946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-08 12:12:32.955162: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Round 0
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.088785045), ('loss', 2.3499544), ('num_examples', 214), ('num_batches', 14)])
	Validation accuracy: 11.66%
Round 1
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.12682927), ('loss', 2.3127558), ('num_examples', 205), ('num_batches', 14)])
	Validation accuracy: 10.90%
Round 2
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.07272727), ('loss', 2.307422), ('num_examples', 220), ('num_batches', 15)])
	Validation accuracy: 10.45%
Round 3
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.13020833), ('loss', 2.2984877), ('num_examples', 192), ('num_batches', 13)])
	Validation accuracy: 13.44%
Round 4
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.15757576), ('loss', 2.288634), ('num_examples', 165), ('num_batches', 12)])
	Validation accuracy: 19.23%
Round 5
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.15668203), ('loss', 2.29675